In [1]:
# get_lof.py
# Usage: get_lof.py FE_plink.chr21.recode.vep.vcf.gz chr21

import sys
import vcf
import time
import logging
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')

logging.warning('Starting. Args={}'.format(sys.argv))



05/09/2021 11:40:45 AM Starting. Args=['/hps/software/users/birney/shimin/miniconda3/envs/r_env/lib/python3.9/site-packages/ipykernel_launcher.py', '-f', '/homes/shimin/.local/share/jupyter/runtime/kernel-e027def3-ada7-4e64-8f12-37ec9ef2c218.json']


In [24]:
sys.argv[1] = '/nfs/research/birney/users/shimin/ukbb/wes/200k/pvcf/4_vep/ukb23156_c10_b0_v1.vep.qc.norm.vcf.bgz'
sys.argv[2] = 'test'

vcf_reader = vcf.Reader(filename=sys.argv[1], compressed=True)
n_record = 0
n_mask1 = 0
n_mask2 = 0

In [3]:
## Find out colnames for CSQ
info_format = vcf_reader.infos['CSQ'].desc.split('Format: ')[1].split('|')
print(info_format)
max_af = info_format.index('MAX_AF')
lof = info_format.index('LoF')
canonical = info_format.index('CANONICAL')
lrt_pred = info_format.index('LRT_pred')
consequence = info_format.index('Consequence')
impact = info_format.index('IMPACT')

['Allele', 'Consequence', 'IMPACT', 'SYMBOL', 'Gene', 'Feature_type', 'Feature', 'BIOTYPE', 'EXON', 'INTRON', 'HGVSc', 'HGVSp', 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 'Existing_variation', 'DISTANCE', 'STRAND', 'FLAGS', 'VARIANT_CLASS', 'SYMBOL_SOURCE', 'HGNC_ID', 'CANONICAL', 'MANE_SELECT', 'MANE_PLUS_CLINICAL', 'TSL', 'APPRIS', 'CCDS', 'ENSP', 'SWISSPROT', 'TREMBL', 'UNIPARC', 'UNIPROT_ISOFORM', 'GENE_PHENO', 'SIFT', 'PolyPhen', 'DOMAINS', 'miRNA', 'HGVS_OFFSET', 'AF', 'AFR_AF', 'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF', 'AA_AF', 'EA_AF', 'gnomAD_AF', 'gnomAD_AFR_AF', 'gnomAD_AMR_AF', 'gnomAD_ASJ_AF', 'gnomAD_EAS_AF', 'gnomAD_FIN_AF', 'gnomAD_NFE_AF', 'gnomAD_OTH_AF', 'gnomAD_SAS_AF', 'MAX_AF', 'MAX_AF_POPS', 'CLIN_SIG', 'SOMATIC', 'PHENO', 'PUBMED', 'MOTIF_NAME', 'MOTIF_POS', 'HIGH_INF_POS', 'MOTIF_SCORE_CHANGE', 'TRANSCRIPTION_FACTORS', 'Ensembl_transcriptid', 'LRT_pred', 'MutationTaster_pred', 'Polyphen2_HDIV_pred', 'Polyphen2_HVAR_pred', 'SIFT_

In [25]:
def apply_mask(csq_records):
    global max_af, lof, canonical, impact, consequence
    for csq in csq_records:
        csq_info = csq.split('|')
        maf = 0 if csq_info[max_af] == '' else float(csq_info[max_af])
        # Rare, canonical
        if csq_info[canonical] == 'YES' and maf < 0.01:
            if csq_info[lof] == 'HC':
                return('m1', csq_info)
            elif csq_info[consequence] == 'missense_variant' and (csq_info[impact] in ('HIGH', 'MODERATE')):
                return('m2', csq_info)
    return(None, csq_info)


# Generate genotype matrix
fgt_m1 = open(sys.argv[2] + '.mask1.genotype.tsv', 'w')
fgt_m2 = open(sys.argv[2] + '.mask2.genotype.tsv', 'w')
# Write header of sample names.
fgt_m1.write("\t".join(['CHROM', 'POS', 'ID', 'REF', 'ALT'] + info_format + vcf_reader.samples) + '\n')
fgt_m2.write("\t".join(['CHROM', 'POS', 'ID', 'REF', 'ALT'] + info_format + vcf_reader.samples) + '\n')

    
# Input VCF with VEP annotated variants
t = time.process_time()
for record in vcf_reader:
    n_record += 1
    if n_record % 1000 == 0:
        delta_t = time.process_time() - t
        logging.warning('Processing {} records, {} secs passed.'.format(n_record, delta_t))
    if record.aaf[0] > 0.01:
        # Not rare in the current cohort, skip
        continue
    mask, info = apply_mask(record.INFO['CSQ'])
    if mask == 'm1':
        n_mask1 += 1
        # The type of genotype. hom_ref = 0 het = 1 hom_alt = 2 uncalled = None
        gt = [str(spl.gt_type) for spl in record.samples]
        fgt_m1.write("\t".join([record.CHROM, str(record.POS), record.ID, record.REF, str(record.ALT[0])] + info + gt) + '\n')
    elif mask == 'm2':
        n_mask2 += 1
        gt = [str(spl.gt_type) for spl in record.samples]
        fgt_m2.write("\t".join([record.CHROM, str(record.POS), record.ID, record.REF, str(record.ALT[0])] + info + gt) + '\n')

fgt_m1.close()
fgt_m2.close()

logging.warning(f'Stats,n_mask1,n_mask2,n_record')
logging.warning(f'Stats,{n_mask1},{n_mask2},{n_record}')

KeyboardInterrupt: 

In [26]:
record.INFO['CSQ']

['T|missense_variant|MODERATE|ZMYND11|ENSG00000015171|Transcript|ENST00000309776|protein_coding|4/14||ENST00000309776.8:c.317C>T|ENSP00000309992.5:p.Thr106Ile|745|317|106|T/I|aCa/aTa|||1||SNV|HGNC|HGNC:16966||||2||CCDS73061.1|ENSP00000309992||B7Z2J6.84|UPI0001914D38||1|tolerated(0.13)|benign(0.006)|Gene3D:1.20.920.10&PANTHER:PTHR46379&SMART:SM00297&CDD:cd05492|||||||||||||||||||||||||||||||ENST00000439456&ENST00000397962&ENST00000381604&ENST00000397959&ENST00000309776&ENST00000509513&ENST00000381591&ENST00000403354&ENST00000381607&ENST00000402736&ENST00000602682&ENST00000397955&ENST00000381584&ENST00000558098&ENST00000627286|D||.&.&.&.&B&.&.&P&.&P&.&.&.&.&.|.&.&.&.&B&.&.&B&.&B&.&.&.&.&.|D&T&.&T&.&T&T&T&T&T&.&T&T&T&.|E9PE09&Q15326&A0A0A0MRY2&Q15326-5&B7Z2J6&Q15326-6&Q15326&B0QZE2&Q15326-2&E7ENI9&Q15326-5&E7EV75&J3QKD2&Q15326-3&J3QKD2|.&.&.&.&.&.&YES&.&.&.&.&.&.&.&.||||',
 'T|missense_variant|MODERATE|ZMYND11|ENSG00000015171|Transcript|ENST00000381584|protein_coding|5/15||ENST00000381584

158